## 第十一章 并发和网络

`时间是大自然防止所有事情同时发生的一种方法.空间是防止所有事情都发生在我身上的方法`

### 并发

* 队列

队列有点像列表:从一头添加事物,从另一头取出事物.这种队列被称为FIFO(先进先出)

* 进程

可以用很多方法实现队列,下面模拟一个洗盘子的人和多个烘干过程

In [3]:
import multiprocessing as mp

def washer(dishes,output):
    for dish in dishes:
        print('Washing',dish,'dish')
        output.put(dish)
        
def dryer(input):
    while True:
        dish = input.get()
        print('Drying',dish,'dish')
        input.task_done()
        
dish_queue = mp.JoinableQueue()
dryer_proc = mp.Process(target=dryer,args=(dish_queue,))
dryer_proc.daemon = True
dryer_proc.start()

dishes = ['salad','bread','entree','dessert']
washer(dishes,dish_queue)
dish_queue.join()

Drying salad dish
Drying bread dish
Drying entree dish
Drying dessert dish
Washing salad dish
Washing bread dish
Washing entree dish
Washing dessert dish


* 线程

In [6]:
# 进程示例
import threading

def do_this(what):
    whoami(what)
    
def whoami(what):
    print('Thread %s says: %s'%(threading.current_thread(),what))
    
if __name__ == '__main__':
    whoami("I'm the main program")
    for n in range(4):
        p = threading.Thread(target=do_this,args=("I'm function %s"%n,))
        p.start()

Thread <_MainThread(MainThread, started 140736963982272)> says: I'm the main program
Thread <Thread(Thread-12, started 123145384677376)> says: I'm function 0
Thread <Thread(Thread-13, started 123145389932544)> says: I'm function 1
Thread <Thread(Thread-14, started 123145384677376)> says: I'm function 2
Thread <Thread(Thread-15, started 123145384677376)> says: I'm function 3


In [8]:
# 用线程改写
import threading, queue
import time

def washer(dishes,dish_queue):
    for dish in dishes:
        print('Washing',dish)
        time.sleep(5)
        dish_queue.put(dish)
        
def dryer(dish_queue):
    while True:
        dish = dish_queue.get()
        print('Drying',dish)
        time.sleep(10)
        dish_queue.task_done()
        
dish_queue = queue.Queue()
for n in range(2):
    dryer_thread = threading.Thread(target=dryer, args=(dish_queue,))
    dryer_thread.start()
dishes = ['salad','bread','entree','desert']
washer(dishes,dish_queue)
dish_queue.join()

Washing salad
Drying salad
Washing bread
DryingWashing  breadentree

WashingDrying  desertentree

Drying desert


### 网络